In [1]:
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
import pickle
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

train algorithm thin this file then make predicitons on scoring side


In [2]:
rf_model = RandomForestClassifier()

In [2]:
train = pd.read_csv('train.csv')

In [3]:
X=pd.DataFrame(train.loc[:, train.columns != 'Survived'])
Y = pd.DataFrame(train['Survived'])

data cleaning

In [4]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Pclass       891 non-null    int64  
 2   Name         891 non-null    object 
 3   Sex          891 non-null    object 
 4   Age          714 non-null    float64
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Ticket       891 non-null    object 
 8   Fare         891 non-null    float64
 9   Cabin        204 non-null    object 
 10  Embarked     889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 76.7+ KB


In [5]:
X['Name'].isnull().sum().sum()

0

In [6]:
X['Age'].isnull().sum().sum()

177

In [7]:
X["Age"].fillna(X['Age'].mode()[0], inplace=True)

In [8]:
X['Age'].isnull().sum().sum()

0

In [9]:
X['Cabin']

0       NaN
1       C85
2       NaN
3      C123
4       NaN
       ... 
886     NaN
887     B42
888     NaN
889    C148
890     NaN
Name: Cabin, Length: 891, dtype: object

In [10]:
X["Cabin"].fillna('No', inplace=True)

In [11]:
X['Cabin'].isnull().sum().sum()

0

In [12]:
X["Embarked"].fillna('No', inplace=True)

In [13]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Pclass       891 non-null    int64  
 2   Name         891 non-null    object 
 3   Sex          891 non-null    object 
 4   Age          891 non-null    float64
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Ticket       891 non-null    object 
 8   Fare         891 non-null    float64
 9   Cabin        891 non-null    object 
 10  Embarked     891 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 76.7+ KB


In [14]:
X.isnull().sum().sum()

0

feature engineering

In [15]:
X

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,No,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,No,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,No,S
...,...,...,...,...,...,...,...,...,...,...,...
886,887,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,No,S
887,888,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,24.0,1,2,W./C. 6607,23.4500,No,S
889,890,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [16]:
X['familymem'] = X['SibSp'] + X['Parch'] + 1

In [17]:
X['alone'] = X['familymem'].map(lambda x: 1 if x<2 else 0)

In [18]:
X.drop(['Name', 'PassengerId', 'Ticket'], inplace=True, axis=1)

In [19]:
X

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,familymem,alone
0,3,male,22.0,1,0,7.2500,No,S,2,0
1,1,female,38.0,1,0,71.2833,C85,C,2,0
2,3,female,26.0,0,0,7.9250,No,S,1,1
3,1,female,35.0,1,0,53.1000,C123,S,2,0
4,3,male,35.0,0,0,8.0500,No,S,1,1
...,...,...,...,...,...,...,...,...,...,...
886,2,male,27.0,0,0,13.0000,No,S,1,1
887,1,female,19.0,0,0,30.0000,B42,S,1,1
888,3,female,24.0,1,2,23.4500,No,S,4,0
889,1,male,26.0,0,0,30.0000,C148,C,1,1


In [20]:
scale = ["Age", "Fare"]

In [21]:
skew_feets=X[scale].skew().sort_values(ascending=False)

In [22]:
skewness=pd.DataFrame({'Skew':skew_feets.astype('float')})

In [23]:
skewness

,Skew
Fare,4.787317
Age,0.657753


In [24]:
skewness=skewness[(skewness.Skew>.75)]

In [25]:
indeces = list(skewness.index)

In [26]:
for x in indeces:
    X[x] = np.log1p(X[x])

In [27]:
Xscale=X[scale]

In [28]:
scaler = MinMaxScaler().fit(Xscale)

In [29]:
pickle.dump(scaler, open("scaler_titanic_pickle.pickle","wb"))

In [30]:
Xscale=pd.DataFrame(scaler.transform(Xscale), columns=Xscale.columns)

In [31]:
encode=list(set(X.columns)-set(scale)-set(['Id']))

In [32]:
Xencode = X[encode]


In [33]:
X=Xscale.merge(Xencode.reset_index(), left_index=True, right_index=True)

In [34]:
X=pd.DataFrame(pd.get_dummies(data=X))

In [35]:
j=X

In [36]:
xtrain, xtest, ytrain , ytest =train_test_split(X,Y, test_size=.3, random_state=0)

In [47]:
xtrain

,Age,Fare,index,SibSp,alone,Parch,Pclass,familymem,Cabin_A10,Cabin_A14,...,Cabin_F4,Cabin_G6,Cabin_No,Cabin_T,Embarked_C,Embarked_No,Embarked_Q,Embarked_S,Sex_female,Sex_male
857,0.635587,0.531333,857,0,1,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,1
52,0.610455,0.697531,52,1,0,0,1,2,0,0,...,0,0,0,0,1,0,0,0,1,0
386,0.007288,0.619959,386,5,0,2,3,8,0,0,...,0,0,1,0,0,0,0,1,0,1
124,0.673285,0.698677,124,0,0,1,1,2,0,0,...,0,0,0,0,0,0,0,1,0,1
578,0.296306,0.438741,578,1,0,0,3,2,0,0,...,0,0,1,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,0.484795,0.710264,835,1,0,1,1,3,0,0,...,0,0,0,0,1,0,0,0,1,0
192,0.233476,0.349451,192,1,0,0,3,2,0,0,...,0,0,1,0,0,0,0,1,1,0
629,0.296306,0.347248,629,0,1,0,3,1,0,0,...,0,0,1,0,0,0,1,0,0,1
559,0.447097,0.466654,559,1,0,0,3,2,0,0,...,0,0,1,0,0,0,0,1,1,0


In [37]:
pickle.dump(xtrain.columns, open("columnssaved_titanic.pickle","wb"))

In [38]:
# ytrain.reset_index(inplace=True)

In [39]:
# ytrain.drop(columns='index', inplace=True)

In [40]:
# ytest.reset_index(inplace=True)

In [41]:
# ytest.drop(columns='index', inplace=True)

In [42]:
# model = LogisticRegression(random_state=0)

In [ ]:
rf_model = LogisticRegression(random_state=0)

In [43]:
model.fit(xtrain,ytrain)

/Users/hannahgraham/Library/Python/3.8/lib/python/site-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/hannahgraham/Library/Python/3.8/lib/python/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=0)

In [44]:
preds=model.predict(xtest)

In [45]:
pickle.dump(model,open("logisticmodel.pickle","wb"))

In [46]:
accuracy_score(ytest, preds)

0.8059701492537313